In [0]:
a = sc.parallelize([1,2,3])
a.collect()

Out[2]: [1, 2, 3]

In [0]:
# df = spark.read.csv("/FileStore/tables/sales_record/sales_record.csv", inferSchema = True, header = True)

In [0]:
df = spark.read.format("csv").option("delimiter",",").option("header","true").option("inferSchema","true").load("/FileStore/tables/sales_record/sales_record.csv")

In [0]:
df.show(n=5, truncate = False)

+---------------------------------+--------------------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|Region                           |Country             |Item Type|Sales Channel|Order Priority|Order Date|Order ID |Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+---------------------------------+--------------------+---------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|Central America and the Caribbean|Antigua and Barbuda |Baby Food|Online       |M             |12/20/2013|957081544|1/11/2014|552       |255.28    |159.42   |140914.56    |87999.84  |52914.72    |
|Central America and the Caribbean|Panama              |Snacks   |Offline      |C             |7/5/2010  |301644504|7/26/2010|2167      |152.58    |97.44    |330640.86    |211152.48 |119488.38   |
|Europe        

In [0]:
df.select("Region","Country","Order ID").show(n=5, truncate = False)

+---------------------------------+--------------------+---------+
|Region                           |Country             |Order ID |
+---------------------------------+--------------------+---------+
|Central America and the Caribbean|Antigua and Barbuda |957081544|
|Central America and the Caribbean|Panama              |301644504|
|Europe                           |Czech Republic      |478051030|
|Asia                             |North Korea         |892599952|
|Asia                             |Sri Lanka           |571902596|
+---------------------------------+--------------------+---------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import count

In [0]:
df.select("Region","Country","Order ID").groupBy("Region","Country").agg(count('Order ID').alias('Num_Sales')).orderBy("Num_Sales", ascending = False).show(n = 20, truncate = False)

+---------------------------------+------------------------+---------+
|Region                           |Country                 |Num_Sales|
+---------------------------------+------------------------+---------+
|Europe                           |San Marino              |40       |
|Europe                           |Andorra                 |40       |
|Sub-Saharan Africa               |Mauritius               |38       |
|North America                    |United States of America|38       |
|Sub-Saharan Africa               |Ghana                   |38       |
|Australia and Oceania            |Tonga                   |37       |
|Central America and the Caribbean|Haiti                   |36       |
|Asia                             |South Korea             |36       |
|Middle East and North Africa     |Somalia                 |36       |
|Central America and the Caribbean|Grenada                 |36       |
|Europe                           |Kosovo                  |35       |
|Austr

In [0]:
df.select("Region","Country","Order ID").groupBy("Region","Country").agg(count('Order ID').alias('Num_Sales')).orderBy("Num_Sales", ascending = False).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [Num_Sales#938L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(Num_Sales#938L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [id=#1409]
      +- HashAggregate(keys=[Region#188, Country#189], functions=[finalmerge_count(merge count#943L) AS count(Order ID#194)#937L])
         +- Exchange hashpartitioning(Region#188, Country#189, 200), ENSURE_REQUIREMENTS, [id=#1406]
            +- HashAggregate(keys=[Region#188, Country#189], functions=[partial_count(Order ID#194) AS count#943L])
               +- FileScan csv [Region#188,Country#189,Order ID#194] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/sales_record/sales_record.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Region:string,Country:string,Order ID:int>




In [0]:
word = "Spark is good. Spark is a big data framework".split(" ")

In [0]:
rdd = sc.parallelize(word, 2)

In [0]:
rdd.collect()

Out[3]: ['Spark', 'is', 'good.', 'Spark', 'is', 'a', 'big', 'data', 'framework']

In [0]:
rdd.distinct().collect()

Out[4]: ['Spark', 'is', 'good.', 'framework', 'a', 'big', 'data']

In [0]:
rdd.filter(lambda x: x!='Spark').collect()

Out[5]: ['is', 'good.', 'is', 'a', 'big', 'data', 'framework']

In [0]:
def wordsStartsWith(word, target):
    return word.startswith(target)

In [0]:
rdd.filter(lambda x: wordsStartsWith(x,'S')).collect()

Out[9]: ['Spark', 'Spark']

In [0]:
num_list = list(range(1,21))

In [0]:
num_list = sc.parallelize(num_list)
num_list.collect()

Out[20]: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [0]:
num_list.map(lambda x:x^2).collect()

Out[21]: [3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13, 18, 19, 16, 17, 22]

In [0]:
num_list.map(lambda x:x*x).collect()

Out[22]: [1,
 4,
 9,
 16,
 25,
 36,
 49,
 64,
 81,
 100,
 121,
 144,
 169,
 196,
 225,
 256,
 289,
 324,
 361,
 400]

In [0]:
rdd.flatMap(lambda x:list(x)).take(10)

Out[23]: ['S', 'p', 'a', 'r', 'k', 'i', 's', 'g', 'o', 'o']

In [0]:
country_rank = (("India",1), ("China",2), ("Brazil",3))
country_rank = sc.parallelize(country_rank)

In [0]:
country_rank = country_rank.map(lambda x: (x[1],x[0]))

country_rank.sortByKey(ascending = False ).collect()

Out[32]: [(3, 'Brazil'), (2, 'China'), (1, 'India')]

In [0]:
num_list.map(lambda x:x*x).reduce(lambda x,y: x+y)

Out[34]: 2870

In [0]:
num_list.map(lambda x:x*x).max()

Out[39]: 400

In [0]:
num_list.map(lambda x:x*x).reduce(lambda x,y: x if x>y else y)

Out[38]: 400

In [0]:
temp = sc.parallelize([59,57.2,53.6,55.4,51.8,53.6,55.4])

In [0]:
fa = temp.map(lambda x: (x-32)*(5/9))
fa.collect()

Out[43]: [15.0,
 14.000000000000002,
 12.000000000000002,
 13.0,
 10.999999999999998,
 12.000000000000002,
 13.0]

In [0]:
fa.filter(lambda x: x>=13).collect()

Out[45]: [15.0, 14.000000000000002, 13.0, 13.0]

In [0]:
data2001list = sc.parallelize(['R1N1', 'R1N2','R1N3', 'R1N4','R1N5', 'R1N6', 'R1N7'])
data2002list = sc.parallelize(['R1N3','R1N4', 'R1N7','R1N8','R1N9'])
data2003list = sc.parallelize(['R1N4','R1N8','R1N10','R1N11','R1N12'])

In [0]:
datalist = data2001list+data2002list+data2003list

In [0]:
datalist.distinct().count()

Out[74]: 12

In [0]:
data2001list.subtract(data2002list).collect()

Out[77]: ['R1N5', 'R1N1', 'R1N2', 'R1N6']

In [0]:
data2001list.union(data2002list).union(data2003list).distinct().collect()

Out[75]: ['R1N4',
 'R1N3',
 'R1N9',
 'R1N12',
 'R1N5',
 'R1N1',
 'R1N2',
 'R1N8',
 'R1N6',
 'R1N11',
 'R1N7',
 'R1N10']

In [0]:
datalist.count()

Out[56]: 16

In [0]:
data2001list.count()

Out[53]: 7

In [0]:
(data2002list+data2003list).count()

Out[55]: 9

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
data = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

In [0]:
schema = StructType([StructField('firstName',StringType(), nullable = True),
                      StructField('middleName',StringType(), nullable = True),
                    StructField('lastName',StringType(), nullable = True),
                    StructField('id',StringType(), nullable = True),
                    StructField('gender',StringType(), nullable = True),
                    StructField('salary',IntegerType(), nullable = True)])

In [0]:
df = spark.createDataFrame(data, schema,)

In [0]:
df.printSchema()

root
 |-- firstName: string (nullable = true)
 |-- middleName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



In [0]:
df.show(truncate=False)

+---------+----------+--------+-----+------+------+
|firstName|middleName|lastName|id   |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+



In [0]:
df = spark.read.csv(path = '/FileStore/tables/Fire_Incidents.csv', header = True, sep = ',' , inferSchema = True)

In [0]:
import pyspark.sql.functions as F
renamed_df = df.select([F.col(col).alias(col.replace(' ', '')) for col in df.columns])


In [0]:
renamed_df.printSchema()

root
 |-- IncidentNumber: double (nullable = true)
 |-- ExposureNumber: double (nullable = true)
 |-- Address: string (nullable = true)
 |-- IncidentDate: string (nullable = true)
 |-- CallNumber: double (nullable = true)
 |-- AlarmDtTm: string (nullable = true)
 |-- ArrivalDtTm: string (nullable = true)
 |-- CloseDtTm: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: double (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: double (nullable = true)
 |-- Box: string (nullable = true)
 |-- SuppressionUnits: double (nullable = true)
 |-- SuppressionPersonnel: double (nullable = true)
 |-- EMSUnits: double (nullable = true)
 |-- EMSPersonnel: double (nullable = true)
 |-- OtherUnits: double (nullable = true)
 |-- OtherPersonnel: double (nullable = true)
 |-- FirstUnitOnScene: string (nullable = true)
 |-- EstimatedPropertyLoss: string (nullable = true)
 |-- EstimatedContentsLoss: string (nullable = true)
 |-- FireFatalities: double (null

In [0]:
renamed_df.select('IncidentNumber','IncidentDate','City').show(10)


+--------------+-------------+-------------+
|IncidentNumber| IncidentDate|         City|
+--------------+-------------+-------------+
|   1.1101416E7|11/1/11      |SF           |
|   1.2077793E7|8/22/12      |SF           |
|     7021713.0|3/15/07      |SF           |
|   1.5065212E7|6/20/15      |San Francisco|
|   1.0097225E7|10/31/10     |SF           |
|     8033299.0|4/17/08      |SF           |
|     8098069.0|11/15/08     |SF           |
|     5008050.0|1/30/05      |SF           |
|     4012483.0|2/13/04      |SF           |
|   1.3110544E7|11/21/13     |SF           |
+--------------+-------------+-------------+



In [0]:
renamed_df.write.parquet('/FileStore/tables/Fire_Incidents.parquet', partitionBy = 'IncidentDate', )

In [0]:
spark.read.parquet('/FileStore/tables/Fire_Incidents.parquet').show(10)

+--------------+--------------+--------------------+------------+--------------+--------------+--------------+-------------+-------+---------+-----------+----+----------------+--------------------+--------+------------+----------+--------------+-------------------+---------------------+---------------------+--------------+------------+------------------+----------------+----------------+--------------------+----------+--------------------+--------------------+------------------+------------------------+--------------------+-------------------+--------------+---------------------+-----------------------+-----------+------------------+----------------------------------+--------------+----------------+--------------------+-----------+--------------+-------------------------------+-----------------------------------+-----------------------------+-------------------------------+-----------------+-------------+------------------+---------------------+---------------------+-------------------

In [0]:
!ls dbfs/FileStore/tables

ls: cannot access 'dbfs/FileStore/tables': No such file or directory


In [0]:
schema = StructType([StructField('Order_ID',dataType = StringType(), nullable = True),
                   StructField('Product',dataType = StringType(), nullable = True),
                   StructField('Quantity_Ordered',dataType = StringType(), nullable = True),
                   StructField('Price_Each',dataType = StringType(), nullable = True),
                   StructField('Order_Date',dataType = StringType(), nullable = True),
                   StructField('Purchase_Address',dataType = StringType(), nullable = True)])

In [0]:
df = spark.read.csv(path = 'FileStore/tables/salesdata/*.csv', schema=schema)

In [0]:
df.show(10)

+--------+--------------------+----------------+----------+--------------+--------------------+
|Order_ID|             Product|Quantity_Ordered|Price_Each|    Order_Date|    Purchase_Address|
+--------+--------------------+----------------+----------+--------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|
|  295665|  Macbook Pro Laptop|               1|      1700|12/30/19 00:01|136 Church St, Ne...|
|  295666|  LG Washing Machine|               1|     600.0|12/29/19 07:03|562 2nd St, New Y...|
|  295667|USB-C Charging Cable|               1|     11.95|12/12/19 18:21|277 Main St, New ...|
|  295668|    27in FHD Monitor|               1|    149.99|12/22/19 15:13|410 6th St, San F...|
|  295669|USB-C Charging Cable|               1|     11.95|12/18/19 12:38|43 Hill St, Atlan...|
|  295670|AA Batteries (4-p...|               1|      3.84|12/31/19 22:58|200 Jefferson St,...|
|  295671|USB-C Charging Cable|         

In [0]:
df.printSchema()

root
 |-- Order_ID: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity_Ordered: string (nullable = true)
 |-- Price_Each: string (nullable = true)
 |-- Order_Date: string (nullable = true)
 |-- Purchase_Address: string (nullable = true)



In [0]:
print((df.count(), len(df.columns)))

(186862, 6)


In [0]:
from pyspark.sql.types import *

In [0]:
schema = StructType([StructField(name = 'id', dataType = IntegerType(), nullable = False),
                    StructField(name = 'first_name', dataType = StringType(), nullable = True),
                    StructField(name = 'last_name', dataType = StringType(), nullable = True),
                     StructField(name = 'fav_movies', dataType = ArrayType(elementType = StringType()), nullable = True),
                    StructField(name = 'salary', dataType = FloatType(), nullable = True),
                    StructField(name = 'image_url', dataType = StringType(), nullable = True),
                    StructField(name = 'date_of_birth', dataType = DateType(), nullable = True),
                    StructField(name = 'active', dataType = BooleanType(), nullable = True),])

In [0]:
df = spark.read.json(path = "FileStore/tables/*.json", schema = schema, multiLine = True)

In [0]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- fav_movies: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- salary: float (nullable = true)
 |-- image_url: string (nullable = true)
 |-- date_of_birth: date (nullable = true)
 |-- active: boolean (nullable = true)



In [0]:
df.show(truncate = False)

+---+----------+-----------+----------------------------------------------------------------------------+-------+-----------------------------------------------+-------------+------+
|id |first_name|last_name  |fav_movies                                                                  |salary |image_url                                      |date_of_birth|active|
+---+----------+-----------+----------------------------------------------------------------------------+-------+-----------------------------------------------+-------------+------+
|1  |Drucy     |Poppy      |[I giorni contati]                                                          |1463.36|http://dummyimage.com/126x166.png/cc0000/ffffff|1991-02-16   |true  |
|2  |Emelyne   |Blaza      |[Musketeer, The, Topralli]                                                  |3006.04|http://dummyimage.com/158x106.bmp/cc0000/ffffff|1991-11-02   |false |
|3  |Max       |Rettie     |[The Forgotten Space, Make It Happen]                    

In [0]:
from pyspark.sql.functions import *

In [0]:
df.select(col("first_name"), col("last_name"), col("date_of_birth")).show(3)

+----------+---------+-------------+
|first_name|last_name|date_of_birth|
+----------+---------+-------------+
|     Drucy|    Poppy|   1991-02-16|
|   Emelyne|    Blaza|   1991-11-02|
|       Max|   Rettie|   1990-03-03|
+----------+---------+-------------+
only showing top 3 rows



In [0]:
df.select(concat_ws(' ', col("first_name"), col('last_name')).alias('full_name'), col('salary'), (col("salary")*0.10+col("salary")).alias("salary_inc")).show()

+------------------+-------+------------------+
|         full_name| salary|        salary_inc|
+------------------+-------+------------------+
|       Drucy Poppy|1463.36|1609.6959838867188|
|     Emelyne Blaza|3006.04|  3306.64404296875|
|        Max Rettie|1422.88|1565.1680053710938|
|       Ilario Kean|3561.36|3917.4961181640624|
|      Toddy Drexel|4934.87|  5428.35712890625|
|   Oswald Petrolli|1153.23| 1268.552978515625|
|     Adrian Clarey|1044.73| 1149.202978515625|
|  Dominica Goodnow|1147.76|1262.5360107421875|
|     Emory Slocomb|1082.11|1190.3209838867188|
|     Jeremias Bode|3472.63|  3819.89287109375|
|    Timothy Ervine|1147.61|1262.3709838867187|
|    Leanora Gooder|1327.02| 1459.722021484375|
|   Claiborn Denham|2623.33|   2885.6630859375|
|  Ambrosi Vidineev|4550.88|  5005.96787109375|
|Feodor Nancekivell|2218.46|  2440.30595703125|
|  Margaux Archbold|1013.75|          1115.125|
|   Balduin Elstone|2302.26|2532.4860107421873|
|    Alfie Hatliffe| 3893.1| 4282.410107

In [0]:
df.select(concat_ws(' ', col("first_name"), col('last_name')).alias('full_name'), col('salary'), (expr("salary*0.10+salary")).alias("salary_inc")).show()
# we can pass in string expression like shown above using expr. It works same as col but is extra powerful when using such expression

+------------------+-------+------------------+
|         full_name| salary|        salary_inc|
+------------------+-------+------------------+
|       Drucy Poppy|1463.36|1609.6959838867188|
|     Emelyne Blaza|3006.04|  3306.64404296875|
|        Max Rettie|1422.88|1565.1680053710938|
|       Ilario Kean|3561.36|3917.4961181640624|
|      Toddy Drexel|4934.87|  5428.35712890625|
|   Oswald Petrolli|1153.23| 1268.552978515625|
|     Adrian Clarey|1044.73| 1149.202978515625|
|  Dominica Goodnow|1147.76|1262.5360107421875|
|     Emory Slocomb|1082.11|1190.3209838867188|
|     Jeremias Bode|3472.63|  3819.89287109375|
|    Timothy Ervine|1147.61|1262.3709838867187|
|    Leanora Gooder|1327.02| 1459.722021484375|
|   Claiborn Denham|2623.33|   2885.6630859375|
|  Ambrosi Vidineev|4550.88|  5005.96787109375|
|Feodor Nancekivell|2218.46|  2440.30595703125|
|  Margaux Archbold|1013.75|          1115.125|
|   Balduin Elstone|2302.26|2532.4860107421873|
|    Alfie Hatliffe| 3893.1| 4282.410107

In [0]:
for i in range(1,10,1):
    print(i)
    df.select(concat_ws(' ', col("first_name"), col('last_name')).alias('full_name'), col('salary'), (expr(f"salary*{i}+salary")).alias("salary_inc")).show(5)
# we can pass in string expression like shown above using expr. It works same as col but is extra powerful when using such expression

1
+-------------+-------+----------+
|    full_name| salary|salary_inc|
+-------------+-------+----------+
|  Drucy Poppy|1463.36|   2926.72|
|Emelyne Blaza|3006.04|   6012.08|
|   Max Rettie|1422.88|   2845.76|
|  Ilario Kean|3561.36|   7122.72|
| Toddy Drexel|4934.87|   9869.74|
+-------------+-------+----------+
only showing top 5 rows

2
+-------------+-------+----------+
|    full_name| salary|salary_inc|
+-------------+-------+----------+
|  Drucy Poppy|1463.36|   4390.08|
|Emelyne Blaza|3006.04|   9018.12|
|   Max Rettie|1422.88|   4268.64|
|  Ilario Kean|3561.36|  10684.08|
| Toddy Drexel|4934.87|  14804.61|
+-------------+-------+----------+
only showing top 5 rows

3
+-------------+-------+----------+
|    full_name| salary|salary_inc|
+-------------+-------+----------+
|  Drucy Poppy|1463.36|   5853.44|
|Emelyne Blaza|3006.04|  12024.16|
|   Max Rettie|1422.88|   5691.52|
|  Ilario Kean|3561.36|  14245.44|
| Toddy Drexel|4934.87|  19739.48|
+-------------+-------+----------+

In [0]:
df.filter("salary > 3000").show(10)

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| id|first_name|last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
|  2|   Emelyne|    Blaza|[Musketeer, The, ...|3006.04|http://dummyimage...|   1991-11-02| false|
|  4|    Ilario|     Kean|[Up Close and Per...|3561.36|http://dummyimage...|   1987-06-09|  true|
|  5|     Toddy|   Drexel|[Walk in the Clou...|4934.87|http://dummyimage...|   1992-10-28|  true|
| 10|  Jeremias|     Bode|[Farewell to Arms...|3472.63|http://dummyimage...|   1997-08-02|  true|
| 14|   Ambrosi| Vidineev|[Wall Street: Mon...|4550.88|http://dummyimage...|   1989-07-20|  true|
| 18|     Alfie| Hatliffe|     [Lord of Tears]| 3893.1|http://dummyimage...|   1989-06-21|  true|
| 19|      Lura|   Follis|[My Life in Pink ...|3331.26|http://dummyimage...|   1998-11-03| false|
| 20|      Maxi|    

In [0]:
df.where("salary > 3000").show(10)

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| id|first_name|last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
|  2|   Emelyne|    Blaza|[Musketeer, The, ...|3006.04|http://dummyimage...|   1991-11-02| false|
|  4|    Ilario|     Kean|[Up Close and Per...|3561.36|http://dummyimage...|   1987-06-09|  true|
|  5|     Toddy|   Drexel|[Walk in the Clou...|4934.87|http://dummyimage...|   1992-10-28|  true|
| 10|  Jeremias|     Bode|[Farewell to Arms...|3472.63|http://dummyimage...|   1997-08-02|  true|
| 14|   Ambrosi| Vidineev|[Wall Street: Mon...|4550.88|http://dummyimage...|   1989-07-20|  true|
| 18|     Alfie| Hatliffe|     [Lord of Tears]| 3893.1|http://dummyimage...|   1989-06-21|  true|
| 19|      Lura|   Follis|[My Life in Pink ...|3331.26|http://dummyimage...|   1998-11-03| false|
| 20|      Maxi|    

In [0]:
df.where((col('salary')>=3000) & (col('salary')<=4000) & (col('active') == False)).show(10)

+---+----------+----------+--------------------+-------+--------------------+-------------+------+
| id|first_name| last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+----------+--------------------+-------+--------------------+-------------+------+
|  2|   Emelyne|     Blaza|[Musketeer, The, ...|3006.04|http://dummyimage...|   1991-11-02| false|
| 19|      Lura|    Follis|[My Life in Pink ...|3331.26|http://dummyimage...|   1998-11-03| false|
| 22|  Theodore|  Climance|[Story of the Wee...|3008.56|http://dummyimage...|   1999-01-30| false|
| 24|  Camellia|  Jervoise| [Waiting to Exhale]|3431.17|http://dummyimage...|   1996-07-14| false|
| 36|     Ozzie|  Brownlie|     [Orange County]|3945.18|http://dummyimage...|   1992-01-25| false|
| 38|    Camile|      Mace|[Family Guy Prese...|3559.93|http://dummyimage...|   1994-12-12| false|
| 48|Maximilian|     Jonin|[The Last Shark, ...|3274.34|http://dummyimage...|   1994-09-16| false|
| 50|    D

In [0]:
df.filter((year("date_of_birth") == 1989) | (year("date_of_birth") == 1999)).show()

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| id|first_name|last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| 14|   Ambrosi| Vidineev|[Wall Street: Mon...|4550.88|http://dummyimage...|   1989-07-20|  true|
| 18|     Alfie| Hatliffe|     [Lord of Tears]| 3893.1|http://dummyimage...|   1989-06-21|  true|
| 22|  Theodore| Climance|[Story of the Wee...|3008.56|http://dummyimage...|   1999-01-30| false|
| 34|     Davis|    Pinks|          [Hounddog]|1337.14|http://dummyimage...|   1989-07-27|  true|
| 49| Kendricks|      Kee|   [Flower & Garnet]|2304.39|http://dummyimage...|   1999-11-14|  true|
| 55|      Jere|Shakelady|[Spencer's Mountain]|2866.28|http://dummyimage...|   1999-05-03| false|
| 61|    Shanna|  Samples|[Thomas in Love (...| 2703.0|http://dummyimage...|   1989-07-07| false|
| 71|    Cecily|  Sa

In [0]:
df.where((year("date_of_birth") == 1989) | (year("date_of_birth") == 1999)).show()

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| id|first_name|last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| 14|   Ambrosi| Vidineev|[Wall Street: Mon...|4550.88|http://dummyimage...|   1989-07-20|  true|
| 18|     Alfie| Hatliffe|     [Lord of Tears]| 3893.1|http://dummyimage...|   1989-06-21|  true|
| 22|  Theodore| Climance|[Story of the Wee...|3008.56|http://dummyimage...|   1999-01-30| false|
| 34|     Davis|    Pinks|          [Hounddog]|1337.14|http://dummyimage...|   1989-07-27|  true|
| 49| Kendricks|      Kee|   [Flower & Garnet]|2304.39|http://dummyimage...|   1999-11-14|  true|
| 55|      Jere|Shakelady|[Spencer's Mountain]|2866.28|http://dummyimage...|   1999-05-03| false|
| 61|    Shanna|  Samples|[Thomas in Love (...| 2703.0|http://dummyimage...|   1989-07-07| false|
| 71|    Cecily|  Sa

In [0]:
df.filter(array_contains('fav_movies', 'Land of the Lost')).show()

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| id|first_name|last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| 11|   Timothy|   Ervine|[Land of the Lost...|1147.61|http://dummyimage...|   1971-06-02| false|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+



In [0]:
df.where(array_contains('fav_movies', 'Land of the Lost')).show()

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| id|first_name|last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| 11|   Timothy|   Ervine|[Land of the Lost...|1147.61|http://dummyimage...|   1971-06-02| false|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+



In [0]:
df.filter(col('salary')>4000).show()

+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| id|first_name|  last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
|  5|     Toddy|     Drexel|[Walk in the Clou...|4934.87|http://dummyimage...|   1992-10-28|  true|
| 14|   Ambrosi|   Vidineev|[Wall Street: Mon...|4550.88|http://dummyimage...|   1989-07-20|  true|
| 20|      Maxi|      Cluet|[All I Want for C...|4046.46|http://dummyimage...|   1979-05-06| false|
| 25|     Kelcy|     Wogdon|    [Iron Mask, The]|4512.51|http://dummyimage...|   2000-10-20|  true|
| 27|    Kelila|Harrowsmith|   [Apparition, The]|4651.58|http://dummyimage...|   1973-01-02|  true|
| 29|       Eli|  Normabell|[Return to Peyton...|4917.48|http://dummyimage...|   1993-01-02|  true|
| 39|   Melinda|   McKevitt|[Safrana or Freed...|4166.13|http://dummyimage...|   1979-12-15|  true|


In [0]:
df.select(col('active')).distinct().show()

+------+
|active|
+------+
|  true|
| false|
|  null|
+------+



In [0]:
df.select(col('first_name'), year(col('date_of_birth')).alias('birth_year'), col('active')).orderBy("birth_year",'first_name').show()

+----------+----------+------+
|first_name|birth_year|active|
+----------+----------+------+
|      null|      null|  null|
|    Adrian|      1971| false|
|   Feodora|      1971|  true|
|       Sky|      1971| false|
|   Timothy|      1971| false|
|    Lucita|      1972|  true|
|      Rodi|      1972| false|
|  Sherline|      1972|  true|
|     Toddy|      1972|  true|
|  Dominica|      1973| false|
|    Kelila|      1973|  true|
|  Wolfgang|      1973|  true|
|   Balduin|      1974| false|
|     Emory|      1974|  true|
|    Norean|      1974|  true|
|    Janean|      1975|  true|
|       Bev|      1976|  true|
| Franciska|      1976| false|
|    Bennie|      1977| false|
|     Johny|      1977| false|
+----------+----------+------+
only showing top 20 rows



In [0]:
df.select(col('first_name'), year(col('date_of_birth')).alias('birth_year'), col('active')).dropDuplicates(["birth_year",'active']).orderBy("birth_year","first_name").show()

+----------+----------+------+
|first_name|birth_year|active|
+----------+----------+------+
|      null|      null|  null|
|    Adrian|      1971| false|
|   Feodora|      1971|  true|
|      Rodi|      1972| false|
|  Sherline|      1972|  true|
|  Dominica|      1973| false|
|    Kelila|      1973|  true|
|   Balduin|      1974| false|
|     Emory|      1974|  true|
|    Janean|      1975|  true|
|       Bev|      1976|  true|
| Franciska|      1976| false|
|     Johny|      1977| false|
|    Daveta|      1978| false|
|   Guthrie|      1978|  true|
|      Maxi|      1979| false|
|   Melinda|      1979|  true|
|    Carter|      1980| false|
|   Loralyn|      1980|  true|
|     Clive|      1981|  true|
+----------+----------+------+
only showing top 20 rows



In [0]:
df.orderBy(desc(year(col('date_of_birth')).alias('year'))).show()

+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| id|first_name|  last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| 92|     Daron|    Briance|[Train on the Bra...|4226.35|http://dummyimage...|   2002-02-22|  true|
| 80|   Lorilee|     Petrie|[Gaudi Afternoon,...|4153.15|http://dummyimage...|   2002-07-29| false|
|100|    Virgie|   Domanski|[Horseman, The, S...|2165.93|http://dummyimage...|   2002-01-05|  true|
| 37|    Carlen|    Sharply|[Dr. Jekyll and M...|2051.85|http://dummyimage...|   2002-06-01|  true|
| 85|    Maxine|    Ewenson|[Savior, Beaver T...| 3363.5|http://dummyimage...|   2001-01-24| false|
| 15|    Feodor|Nancekivell|   [Monsoon Wedding]|2218.46|http://dummyimage...|   2000-10-07| false|
| 69|  Annabell|    Doughty|[Entertaining Ang...|2022.57|http://dummyimage...|   2000-09-03|  true|


In [0]:
df.orderBy(year(col('date_of_birth')).alias('birth_year'), ascending = False).show()

+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| id|first_name|  last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| 80|   Lorilee|     Petrie|[Gaudi Afternoon,...|4153.15|http://dummyimage...|   2002-07-29| false|
|100|    Virgie|   Domanski|[Horseman, The, S...|2165.93|http://dummyimage...|   2002-01-05|  true|
| 37|    Carlen|    Sharply|[Dr. Jekyll and M...|2051.85|http://dummyimage...|   2002-06-01|  true|
| 92|     Daron|    Briance|[Train on the Bra...|4226.35|http://dummyimage...|   2002-02-22|  true|
| 85|    Maxine|    Ewenson|[Savior, Beaver T...| 3363.5|http://dummyimage...|   2001-01-24| false|
| 15|    Feodor|Nancekivell|   [Monsoon Wedding]|2218.46|http://dummyimage...|   2000-10-07| false|
| 69|  Annabell|    Doughty|[Entertaining Ang...|2022.57|http://dummyimage...|   2000-09-03|  true|
